<a href="https://colab.research.google.com/github/oserikov/data-science-nlp/blob/master/ngrams_%D0%B8_%D0%BC%D0%B0%D1%80%D0%BA%D0%BE%D0%B2%D1%81%D0%BA%D0%B0%D1%8F_%D1%86%D0%B5%D0%BF%D1%8C_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Возьмём какой-нибудь корпус. Пусть тот же "полярный круг" с прошлого занятия

In [0]:
!wget https://raw.githubusercontent.com/oserikov/data-science-nlp/master/data/polkrug.txt

--2019-10-31 19:59:53--  https://raw.githubusercontent.com/oserikov/data-science-nlp/master/data/polkrug.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18547851 (18M) [text/plain]
Saving to: ‘polkrug.txt’

polkrug.txt         100%[===================>]  17.69M  --.-KB/s    in 0.1s    

2019-10-31 19:59:54 (183 MB/s) - ‘polkrug.txt’ saved [18547851/18547851]



In [0]:
polkrug_texts = [line.strip() 
                 for line in open("polkrug.txt", 'r', encoding="utf-8")
                 if not line.isspace()]

In [0]:
import re

In [0]:
punct = '[!"#$%&()*+,./:;<=>?@[\]^_`{|}~„“«»†*/\—–‘’]'

polkrug_texts = [re.sub(punct, '', text) for text in polkrug_texts]

Посчитаем частоты n-грамм

In [0]:
from collections import defaultdict as dd

def log_before(msg):
    print(f"BEFORE {msg}")

def log_after(msg):
    print(f"AFTER {msg}")

SENT_START_TOKEN = "<START>"
SENT_END_TOKEN = "<END>"


ngrams_freqs = dd(int)

NGRAMS_LEN_LOWER = 1
NGRAMS_LEN_UPPER = 5

LOG_FREQ = 100
for text_idx, text in enumerate(polkrug_texts):
    if text_idx % LOG_FREQ == 0:
        log_before(f"processing text {text_idx} of {str(len(polkrug_texts))}")
    
    text_words = [SENT_START_TOKEN] + text.split() + [SENT_END_TOKEN]
    for ngram_len in range(NGRAMS_LEN_LOWER, NGRAMS_LEN_UPPER + 1):
        for ngram_start_idx in range(len(text_words) - ngram_len):
            ngram_end_idx = ngram_start_idx + ngram_len
            curr_ngram = text_words[ngram_start_idx: ngram_end_idx]
            ngrams_freqs[tuple(curr_ngram)] += 1
    
    if text_idx % LOG_FREQ == 0:
        log_after(f"processing text {text_idx} of {str(len(polkrug_texts))}")
    

BEFORE processing text 0 of 35467
AFTER processing text 0 of 35467
BEFORE processing text 100 of 35467
AFTER processing text 100 of 35467
BEFORE processing text 200 of 35467
AFTER processing text 200 of 35467
BEFORE processing text 300 of 35467
AFTER processing text 300 of 35467
BEFORE processing text 400 of 35467
AFTER processing text 400 of 35467
BEFORE processing text 500 of 35467
AFTER processing text 500 of 35467
BEFORE processing text 600 of 35467
AFTER processing text 600 of 35467
BEFORE processing text 700 of 35467
AFTER processing text 700 of 35467
BEFORE processing text 800 of 35467
AFTER processing text 800 of 35467
BEFORE processing text 900 of 35467
AFTER processing text 900 of 35467
BEFORE processing text 1000 of 35467
AFTER processing text 1000 of 35467
BEFORE processing text 1100 of 35467
AFTER processing text 1100 of 35467
BEFORE processing text 1200 of 35467
AFTER processing text 1200 of 35467
BEFORE processing text 1300 of 35467
AFTER processing text 1300 of 35467
BE

Посчитаем условные вероятности для цепочек соответствущих длин

In [0]:
cond_seq2w_probs = dd(lambda : dd(lambda : dd(int)))
for ngram in ngrams_freqs.keys():
    if len(ngram) == 1:
        continue
    else:
        ngram_freq = ngrams_freqs[ngram]

        preceding_sequence, tail_word = ngram[:-1], ngram[-1]
        preceding_sequence_freq = ngrams_freqs[preceding_sequence]
        
        cond_proba = ngram_freq / preceding_sequence_freq
        cond_seq2w_probs[preceding_sequence][tail_word] = cond_proba

In [0]:
import random

In [0]:
def naive_sample(ngram_to_sample_len = NGRAMS_LEN_UPPER -1):
    start_seq = (SENT_START_TOKEN,)
    cur_seq = start_seq
    while cur_seq[-1] != SENT_END_TOKEN:
        prefix_to_sample_for = cur_seq[- min(len(cur_seq), ngram_to_sample_len):]
        if prefix_to_sample_for not in cond_seq2w_probs.keys():
            # nothing to proceed with
            cur_seq += (SENT_END_TOKEN,)
            break

        cur_seq += tuple(random.choices(list(cond_seq2w_probs[prefix_to_sample_for].keys()),
                                        list(cond_seq2w_probs[prefix_to_sample_for].values())))
    
    return cur_seq

In [0]:
while True:
    sample = naive_sample()
    print(sample)
    input("proceed?: ")

('<START>', 'Основным', 'направлением', 'деятельности', 'коллектива', 'детского', 'сада', 'будет', 'реализация', 'здоровьесберегающих', 'технологий', 'Для', 'проведения', 'оздоровительных', 'процедур', 'кроме', 'медицинских', 'кабинетов', 'есть', 'спелеокамера', 'Она', 'предназначена', 'для', 'профилактики', 'лечения', 'и', 'реабилитации', 'заболеваний', 'дыхательной', 'сердечно-сосудистой', 'и', 'нервной', 'систем', 'аллергозов', 'у', 'взрослых', 'и', 'детей', 'Стены', 'и', 'пол', 'спелеокамеры', 'выполнены', 'из', 'сильвинито-галитовой', 'соли', 'Верхнекамского', 'месторождения', 'Западного', 'Урала', '<END>')
proceed?: 
('<START>', 'Нет', 'комментариев', '<END>')


KeyboardInterrupt: ignored

In [0]:
def naive_score(sequence, ngram_len):
    # almost markov:
    # multiply all the ngrams probas
    # approximate first n-2 tokens probas with the longest possible ngrams
    score = 1
    for idx in range(2, len(sequence)):
        cur_subsequence = sequence[:idx]
        cur_suffix_to_score = cur_subsequence[- min(len(cur_subsequence), ngram_len):]
        
        subsequence_score = cond_seq2w_probs[cur_suffix_to_score[:-1]][cur_suffix_to_score[-1]]
        if not isinstance(subsequence_score, float):
            subsequence_score = 0
        score *= subsequence_score
    return score


In [0]:
ngram_len = 2
sample_seq = naive_sample(ngram_len)
print(naive_score(sample_seq, ngram_len), sample_seq)

3.1537982797578206e-223 ('<START>', 'Администрация', 'Салехарда', 'проводит', 'два', 'раза', 'Из', '1403', 'человек', 'живущих', 'в', 'нашем', 'городе', 'созданы', 'необходимые', 'условия', 'для', 'получения', 'выплаты', 'в', 'рамках', 'проекта', 'проходят', 'встречи', 'населения', 'с', '1', 'января', '2012', 'года', 'До', 'этого', 'около', 'года', 'Здесь', 'отмечают', 'Новый', 'год', 'которая', 'стартовала', 'после', 'обеда', 'и', 'после', 'службы', 'в', 'армии', 'попали', 'в', 'плен', 'бежала', 'и', 'бежала', 'В', 'тот', 'день', 'глава', 'большого', 'семейства', 'много', 'шутил', 'рассказывая', 'гостям', 'истории', 'произошедшие', 'с', 'ним', 'ЕГРП', 'и', 'государственного', 'имущества', 'стали', 'нормой', 'жизни', 'Желаю', 'дальнейшего', 'совершенствования', 'профессионального', 'мастерства', 'даёт', 'возможность', 'доставки', 'российской', 'арктической', 'продукции', 'напрямую', 'заказчикам', 'в', 'странах', 'Азиатско-Тихоокеанского', 'региона', 'Ледокол', 'Арктика', 'станет', 'сам